# EDA

load data

In [63]:
import os
import pandas as pd

data_folder = '../data/train'
img_files = os.listdir(data_folder)

labels = pd.read_csv('../data/trainLabels.csv')
labels

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
...,...,...
35121,44347_right,0
35122,44348_left,0
35123,44348_right,0
35124,44349_left,0


make sure train imgs/labels match

In [64]:
assert len(img_files) == len(labels)
for i in labels['image']:
    assert os.path.exists(os.path.join(data_folder, str(i) + '.jpeg'))

# EDA Images

TODO:
* image resolutions --> DONE (can also find median/quartiles if we want)
* describe pixel values/ranges --> takes too long, ~20min
* split into train/val/test (70/10/20%) but ensure right/left images do not leak across train/val/test
* normalize/preproc?


describe image size/resolution

In [65]:
from PIL import Image
import numpy as np

dims = []

for i in img_files:
    img = Image.open(os.path.join(data_folder, i))
    dims.append(img.size)

dims = np.array(dims)
print('min size:', np.min(dims, axis=0))
print('max size:', np.max(dims, axis=0))
print('mean size:', np.mean(dims, axis=0))
print('std size:', np.std(dims, axis=0))

min size: [400 289]
max size: [5184 3456]
mean size: [3636.65524113 2473.03316632]
std size: [801.81875595 471.46775397]


# EDA Labels

TODO:
* nan values? --> DONE
* label freqs --> DONE
* label breakdown by right vs left eye?

convert level to class label

In [66]:
labels
labels_map = {0: '0-No DR', 1: '1-Mild', 2: '2-Moderate', 3: '3-Severe', 4: '4-Proliferative DR'}  
labels['label'] = labels['level'].map(labels_map)
labels

,image,level,label
0,10_left,0,0-No DR
1,10_right,0,0-No DR
2,13_left,0,0-No DR
3,13_right,0,0-No DR
4,15_left,1,1-Mild
...,...,...,...
35121,44347_right,0,0-No DR
35122,44348_left,0,0-No DR
35123,44348_right,0,0-No DR
35124,44349_left,0,0-No DR


check if there are any nan values

In [67]:
labels.isna().sum()

image    0
level    0
label    0
dtype: int64

print label frequencies

In [68]:
# label frequencies
print(labels['label'].value_counts().sort_index())
print()
print(labels['label'].value_counts(normalize=True).sort_index())

label
0-No DR               25810
1-Mild                 2443
2-Moderate             5292
3-Severe                873
4-Proliferative DR      708
Name: count, dtype: int64

label
0-No DR               0.734783
1-Mild                0.069550
2-Moderate            0.150658
3-Severe              0.024853
4-Proliferative DR    0.020156
Name: proportion, dtype: float64
